In [ ]:
from grid_world import RelayConfig, ClientConfig, InitConfig
import numpy as np
from loguru import logger
from IPython.display import clear_output
import time
from gymnasium.envs.registration import register
import gymnasium as gym

size = 1000
relay_config = RelayConfig(num=10, speed=10.0, limit_position=True, limit_height=True, max_height=100.0, min_height=50.0)
client_config = ClientConfig(num=100, speed=5.0, traffic=2.0, link_establish=2.0, is_move=False)
init_config = InitConfig(center_type="center", relay_type="follow", client_type="random")

# register the environment
register(
    id='GridWorld-v0',
    entry_point='grid_world:GridWorldEnv',
    max_episode_steps=500,
    kwargs={
        "size": size,
        "relay_config": relay_config,
        "client_config": client_config,
        "init_config": init_config,
        "is_polar": False,
        "is_plot": True,
        "is_log": True,
        "use_model": False
    }
)

# create the environment
env = gym.make('GridWorld-v0')

seed = 0
# Set seeds
env.action_space.seed(seed)
np.random.seed(seed)
state, info = env.reset(seed=seed)
check_relay_location = False
check_client_location = False
check_center_location = False
check_state = False


for i in range(100):
    
    action = env.action_space.sample()
    # action = np.zeros_like(action)
    # action[:, 2] = 10

    if check_relay_location:
        # check whether the relay's location is changed
        old_relay_value = state["relay"]

    if check_client_location:
        # check whether the client's location is changed
        old_client_value = state["client"]

    if check_center_location:
        # check whether the client's location is changed
        old_center_value = state["center"]

    next_state, reward, terminated, truncated, info = env.step(action=action) 
    logger.info(f"reward:{reward}")

    state = next_state

    if check_relay_location:
        # check whether the relay's location is changed
        new_relay_value = state["relay"]
        
        if np.all(new_relay_value == old_relay_value):
            # the relay's location does not change
            logger.debug("the relay's location does not change")
        elif np.all(np.abs(new_relay_value - old_relay_value) < relay_config.speed):
            # the relay's location changes normally
            logger.info("the relay's location changes normally")
        else:
            # the relay's location changes too much
            logger.warning("the relay's location changes too much")

    if check_client_location:
        # check whether the client's location is changed
        new_client_value = info["client"]

        if np.all(new_client_value < 0) or np.all(new_client_value > size):
            # the client's location is out of the grid
            logger.warning("the client's location is out of the grid")

        if np.all(old_client_value==new_client_value):
            # the client's location does not change
            logger.warning("the client's location does not change")
        elif np.all(np.abs(new_client_value - old_client_value) < client_config.speed):
            # the client's location changes normally
            logger.info("the client's location changes normally")
        else:
            # the client's location changes too much
            logger.warning("the client's location changes too much")

    if check_center_location:
        # check whether the client's location is changed
        new_center_value = info["center"]

        if np.all(new_center_value < 0) or np.all(new_center_value > size):
            # the center's location is out of the grid
            logger.warning("the center's location is out of the grid")

        if np.all(old_center_value==new_center_value):
            # the center's location does not change
            logger.info("the center's location stays the same")
        else:
            # the center's location changes
            logger.warning("the center's location changes")

    
    time.sleep(0.2)
    clear_output(wait=True)

In [ ]:
print(info.keys())

print(env.action_space)

In [ ]:
# print relay_traffic
relay_traffic = info["relay_traffic"]
logger.info(f"relay_traffic: {[f"{traffic:.2f}" for traffic in relay_traffic]}")
# print relay_center_link_speed_list
relay_center_link_speed_list = info["relay_center_link_speed_list"]
logger.info(f"relay_center_link_speed: {[f"{relay_center_link_speed:.2f}" for relay_center_link_speed in relay_center_link_speed_list]}")
relay_index = np.where(relay_center_link_speed_list >0)
logger.info(f"relay link center id: {relay_index}")
logger.info(f"relay link center speed: {relay_center_link_speed_list[relay_index]}")

In [ ]:
# print relay_distance_matrix
relay_distance_matrix = info["relay_distance_matrix"]
for i in range(relay_distance_matrix.shape[0]):
        logger.info(f"node {i} distance list: {[f"{distance:.2f}" for distance in relay_distance_matrix[i]]}")

In [ ]:
# print relay_center_distance_list
relay_center_distance = info["relay_center_distance"]
logger.info(f"relay center distance: {[f"{distance:.2f}" for distance in relay_center_distance]}")
logger.info(f"relay order: {np.argsort(relay_center_distance)}")

In [ ]:
# print relay_link_speed_matrix
relay_link_speed_matrix = info["relay_link_speed_matrix"]
for i in range(relay_link_speed_matrix.shape[0]):
        logger.info(f"node {i} link speed list: {[f"{link_speed:.2f}" for link_speed in relay_link_speed_matrix[i]]}")

In [ ]:
print(state)